In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

# Data preparation

In [2]:
data = pd.read_csv('train_test.csv')

In [3]:
drop_cols = ['charttime','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','los_icu','text_embeddings',
             'gender','admission_age','weight_admit','height','admission_type','charlson_score','atrial_fibrillation',
             'malignant_cancer','chf','ckd','cld','copd','diabetes','hypertension','ihd','stroke','los_icu_class']
data = data.drop(columns=drop_cols)

In [4]:
# fill the nan
df_full = data.groupby('id').transform(lambda x: x.fillna(x.mean()))
df = df_full.fillna(df_full.mean())

In [5]:
X = df.drop(columns=['icu_death'])
y = df['icu_death']

time_steps = 25
X = np.array([X[i:i + time_steps] for i in range(0, len(X), time_steps)])
y = np.array([y[i] for i in range(time_steps - 1, len(y), time_steps)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_train_2d = X_train.reshape(-1, X_train.shape[-1])
X_test_2d = X_test.reshape(-1, X_test.shape[-1])

In [7]:
# min-max standardization
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train_2d)
X_train_2d_scaled = scaler.transform(X_train_2d)
X_test_2d_scaled = scaler.transform(X_test_2d)

In [8]:
X_train = np.array([X_train_2d_scaled[i:i + time_steps] for i in range(0, len(X_train_2d_scaled), time_steps)])
X_test = np.array([X_test_2d_scaled[i:i + time_steps] for i in range(0, len(X_test_2d_scaled), time_steps)])
y_train = y_train.reshape(-1, 1)

# Modeling

In [11]:
# get the time stamp and num of features
time_steps = X_train.shape[1]
num_features = X_train.shape[2]

# tcn model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(time_steps, num_features)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1, activation='sigmoid')  # output layer, bi-classification
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.F1Score()])


model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

Epoch 1/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - f1_score: 0.1908 - loss: 0.3606 - val_f1_score: 0.1779 - val_loss: 0.3072
Epoch 2/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - f1_score: 0.1916 - loss: 0.3187 - val_f1_score: 0.1779 - val_loss: 0.2921
Epoch 3/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - f1_score: 0.2001 - loss: 0.3133 - val_f1_score: 0.1779 - val_loss: 0.2904
Epoch 4/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - f1_score: 0.1936 - loss: 0.3016 - val_f1_score: 0.1779 - val_loss: 0.2895
Epoch 5/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - f1_score: 0.1950 - loss: 0.2981 - val_f1_score: 0.1779 - val_loss: 0.3007
Epoch 6/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - f1_score: 0.2035 - loss: 0.3054 - val_f1_score: 0.1779 - val_loss: 0.2901
Epoch 7/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - f1_score: 0.1925 - loss: 0.2903 - val_f1_score: 0.1779 - val_loss: 0.2880
Epoch 8/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - f1_score: 0.1985 - loss: 0.3023 - val_f1_score: 0.

In [ ]:
# # save the trained model
# model.save('tcn_mortality.keras')

In [12]:
# on testing set
y_test_pred = model.predict(X_test)

# classify based on probability
y_pred_class = (y_test_pred > 0.5).astype(int)

# evaluate the result
auroc = roc_auc_score(y_test, y_pred_class)
precision = precision_score(y_test, y_pred_class)
recall = recall_score(y_test, y_pred_class)
f1 = f1_score(y_test, y_pred_class)

print("AUROC:", auroc)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
AUROC: 0.6019909036776339
Precision: 0.3452914798206278
Recall: 0.2532894736842105
F1 Score: 0.2922201138519924


In [ ]:
# # predict the whole X and output probability
# y_pred = model.predict(X_scaled)

# result= pd.DataFrame(y_pred, columns=['probs'])
# result.to_csv('tcn_probs_of_death_traintest.csv', index=False)

511/511 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
